In [6]:
import xml.etree.ElementTree as ET

def load_external_ids(filename):
    external_id_dict = {}
    with open(filename, "r") as file:
        for line in file:
            left, right = line.split(" : ")
            left = left.strip()
            right = right.strip()
            external_id_dict[left] = right
    return external_id_dict

def get_last_timestep_time_for_external_id(xml_file, external_id):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    last_time = None

    for timestep in root.findall('timestep'):
        time = float(timestep.get('time'))
        if any(vehicle.get('id') == external_id for vehicle in timestep.findall('vehicle')):
            last_time = time

    return last_time

def get_seqs_before_last_time(filename, last_time):
    seq_list = []
    with open(filename, "r") as file:
        for line in file:
            time, seq = line.strip().split(":")
            time = float(time)
            seq = int(seq)
            
            if time < last_time:
                seq_list.append(seq)
                
    return seq_list

def calculate_pdr(send_seqs, reply_seqs):
    total_sent = len(send_seqs)
    
    successfully_received = len([seq for seq in send_seqs if seq in reply_seqs])
    
    if total_sent == 0:
        return 0
    else:
        pdr = successfully_received / total_sent
        return pdr

def analyze_pdr_for_node(xml_file, seedType, node_number):
    external_id_dict = load_external_ids(f'external_id_table (Seed {seedType}).txt')
    external_id = external_id_dict.get(str(node_number))
    
    if external_id:
        last_time = get_last_timestep_time_for_external_id(xml_file, external_id)
        if last_time is not None:
            print(f"Node {node_number}에 해당하는 external_id {external_id}의 마지막 time 값은 {last_time}입니다.")
            
            send_filename = f"PingSeq (Seed {seedType})/Send{node_number}{aodvType}.txt"
            reply_filename = f"PingSeq (Seed {seedType})/Reply{node_number}{aodvType}.txt"
            
            send_seqs = get_seqs_before_last_time(send_filename, last_time)
            reply_seqs = get_seqs_before_last_time(reply_filename, last_time)

            pdr = calculate_pdr(send_seqs, reply_seqs)
            print(f"Sent : {len(send_seqs)}\t\t Received : {len(reply_seqs)}\t\t PDR {node_number} : {pdr * 100:.2f}%")
        else:
            print(f"Node {node_number}에 해당하는 external_id {external_id}의 timestep 정보를 찾을 수 없습니다.")
    else:
        print(f"Node {node_number}에 해당하는 external_id를 찾을 수 없습니다.")

xml_file = 'fcd/map2.xml'
external_id_file = "external_id_table.txt"
node_number = 22
aodvType = 'O'
seedType = '2'
analyze_pdr_for_node(xml_file, seedType, node_number)


Node 22에 해당하는 external_id 25의 마지막 time 값은 199.9입니다.
Sent : 700		 Received : 403		 PDR 22 : 57.57%
